In [10]:
from enum import IntEnum, Enum, auto

class XState(IntEnum): # Session Termination
    BI_DIRECTIONAL = 0
    LNMD_LOCAL = 1
    SENDING_RNMD = 2
    GOT_RNMD_HAVE_DATA = 3
    GOT_RNMD_WAIT_LOCAL_EMPTY = 4
    BOTH_EMPTY = 5

class YState(IntEnum): # Comm Change
    IDLE = 0
    LOCAL_COMM_CHANGE = 1
    SENDING_COMM_CHANGE = 2
    WAIT_ACK = 3
    RCCD_RECEIVED = 4 # full duplex only
    ACT_ON_RCCD = 5 # full duplex only

class ZState(IntEnum): # Symbol Inlock Status
    INLOCK_TRUE = 0
    INLOCK_FALSE = 1

class ModulationState(Enum):
    OFF = 0
    ON = 1

class BoolVar(Enum):
    FALSE = 0
    TRUE = 1

""""
class Prox1ControlState: # prototype state struct
    def __init__(self):
        # Session Termination / Comm_Change
        self.X = XState.BI_DIRECTIONAL
        self.Y = YState.IDLE
        self.Z = ZState.INLOCK_TRUE

        # Physical interface
        self.MODULATION = ModulationState.OFF

        # PLCW / status report
        self.NEED_PLCW = BoolVar.TRUE
        self.NEED_STATUS_REPORT = BoolVar.TRUE

        # Buffers
        self.REMOTE_SCID_BUFFER = 0
        self.COMMUNICATION_VALUE_BUFFER = None
        self.RECEIVING_SCID_BUFFER = None
        self.REMOTE_PCID_BUFFER = None

    # Session termination around X
    # Track the sub-states of the full and half-duplex session termination process
    # Tracks where you are in closing a data session between two transceivers

    def local_no_more_data(self, duplex="full"):
        if duplex == "half":
            # in half duplex, X = 1 means local LNMD
            if self.X == XState.BI_DIRECTIONAL:
                self.X = XState.LMND_LOCAL
        else:
            # in full duplex, we may directly proceed to sending RNMD
            if self.X in (XState.BI_DIRECTIONAL, XState.GOT_RNMD_WAIT_LOCAL_EMPTY):
                self.X = XState.SENDING_RNMD

    # Local sends RNMD across the link
    def send_rnmd(self):
        if self.X in (XState.LMND_LOCAL, XState.BI_DIRECTIONAL):
            self.X = XState.SENDING_RNMD

    # Receive RNMD from remote
    def receive_rnmd(self, local_has_data: bool):
        if local_has_data:
            # half duplex: remote has no data, local still has some
            self.X = XState.GOT_RNMD_HAVE_DATA
        else:
            # both are empty now
            self.X = XState.BOTH_EMPTY

    # When both empty, terminate and restart
    def terminate_session_if_done(self):
        if self.X == XState.BOTH_EMPTY:
            # inform "vehicle controller" here in a real system
            self.X = XState.BI_DIRECTIONAL

    # Comm change around Y
    # Track the sub-states during the commanding of a physical layer communications change
    # Tracks where you are in changing the physical-layer communication settings

    def request_comm_change(self, new_comm_value):
        # step 1: higher layer asks for comm_change
        self.Y = YState.LOCAL_COMM_CHANGE
        self.COMMUNICATION_VALUE_BUFFER = new_comm_value

    def start_sending_comm_change(self):
        self.Y = YState.SENDING_COMM_CHANGE

    def comm_change_sent_wait_ack(self):
        if self.Y == YState.SENDING_COMM_CHANGE:
            self.Y = YState.WAIT_ACK

    def receive_comm_change_ack(self):
        if self.Y == YState.WAIT_ACK:
            self.Y == YState.IDLE

    # Z is updated by lower layer's lock status
    # Tracks the nondeterministic events
    # (losing lock can happen at unpredictable moments, so Z remembers if it has already happened)
    # tracks whether the receiver has lost symbol lock during a comm_change state

    def symbol_inlock_changed(self, in_lock: bool):
        self.Z = ZState.INLOCK_TRUE if in_lock else ZState.INLOCK_FALSE

    # for modulation and PLCW / status report

    def set_modulation(self, on: bool):
        self.modulation = ModulationState.ON if on else ModulationState.OFF

    def mark_need_plcw(self):
        self.NEED_PLCW = BoolVar.TRUE

    def mark_need_status_report(self):
        self.NEED_STATUS_REPORT = BoolVar.TRUE

    def select_plcw_for_output(self):
        # called by "data selection for output"
        if self.NEED_PLCW == BoolVar.TRUE:
            self.NEED_PLCW = BoolVar.FALSE
            return True
        return False

    def select_status_report_for_output(self):
        if self.NEED_STATUS_REPORT == BoolVar.TRUE:
            self.NEED_STATUS_REPORT = BoolVar.FALSE
            return True
        return False

    # for SCID/PCID buffers, keep simple setters and compare helpers

    def load_remote_scid(self, scid: int):
        assert 0 <= scid <= 1023
        self.REMOTE_SCID_BUFFER = scid

    def load_receiving_scid(self, scid: int):
        assert 0 <= scid <= 1023
        self.RECEIVING_SCID_BUFFER = scid

    def load_receiving_pcid(self, pcid: int):
        self.RECEIVING_PCID_BUFFER = pcid

    def validate_frame_ids(self, frame_scid: int, frame_pcid: int) -> bool:
        scid_ok = (
            self.RECEIVING_SCID_BUFFER is None
            or frame_scid == self.RECEIVING_SCID_BUFFER
        )
        pcid_ok = (
            self.RECEIVING_PCID_BUFFER is None
            or frame_pcid == self.RECEIVING_PCID_BUFFER
        )
        return scid_ok and pcid_ok
"""

'"\nclass Prox1ControlState: # prototype state struct\n    def __init__(self):\n        # Session Termination / Comm_Change\n        self.X = XState.BI_DIRECTIONAL\n        self.Y = YState.IDLE\n        self.Z = ZState.INLOCK_TRUE\n\n        # Physical interface\n        self.MODULATION = ModulationState.OFF\n\n        # PLCW / status report\n        self.NEED_PLCW = BoolVar.TRUE\n        self.NEED_STATUS_REPORT = BoolVar.TRUE\n\n        # Buffers\n        self.REMOTE_SCID_BUFFER = 0\n        self.COMMUNICATION_VALUE_BUFFER = None\n        self.RECEIVING_SCID_BUFFER = None\n        self.REMOTE_PCID_BUFFER = None\n\n    # Session termination around X\n    # Track the sub-states of the full and half-duplex session termination process\n    # Tracks where you are in closing a data session between two transceivers\n\n    def local_no_more_data(self, duplex="full"):\n        if duplex == "half":\n            # in half duplex, X = 1 means local LNMD\n            if self.X == XState.BI_DIRECTI

In [11]:
"""
def demo_full_duplex_termination():
    state = Prox1ControlState()

    # both sides with bi-directional transfer
    print("Initial X:", state.X)

    # local runs out of data
    state.local_no_more_data(duplex="full")
    print("After LMND local:", state.X)

    # remote replies with RNMD; assume no local data
    state.receive_rnmd(local_has_data=False)
    print("After receive RNMD:", state.X)

    # finalize termination
    state.terminate_session_if_done()
    print("After termination:", state.X)

if __name__ == "__main__":
    demo_full_duplex_termination()

"""

'\ndef demo_full_duplex_termination():\n    state = Prox1ControlState()\n\n    # both sides with bi-directional transfer\n    print("Initial X:", state.X)\n\n    # local runs out of data\n    state.local_no_more_data(duplex="full")\n    print("After LMND local:", state.X)\n\n    # remote replies with RNMD; assume no local data\n    state.receive_rnmd(local_has_data=False)\n    print("After receive RNMD:", state.X)\n\n    # finalize termination\n    state.terminate_session_if_done()\n    print("After termination:", state.X)\n\nif __name__ == "__main__":\n    demo_full_duplex_termination()\n\n'

In [12]:
# -------- DECODER HELPERS: TURN VALUES INTO TEXT --------

def describe_x(x: XState, duplex: str) -> str:
    if x == XState.BI_DIRECTIONAL:
        return "X=0: Bi-directional data passing in progress; neither side has declared no-more-data."
    if x == XState.LNMD_LOCAL:
        return "X=1: Local has no more data (LNMD), but this is a half-duplex-only sub-state."
    if x == XState.SENDING_RNMD:
        return "X=2: Local has accepted LNMD and is sending RNMD to the remote; termination will start once RNMD is exchanged."
    if x == XState.GOT_RNMD_HAVE_DATA:
        return "X=3: Local still has data, but it has received RNMD from the remote (half-duplex only)."
    if x == XState.GOT_RNMD_WAIT_LOCAL_EMPTY:
        return "X=4: Local has received RNMD; when local runs out of data it will send RNMD back and complete termination."
    if x == XState.BOTH_EMPTY:
        return "X=5: Both local and remote have no more data; after RNMD exchange the session terminates and X resets to 0."
    return "Unknown X value."

def describe_y(y: YState, duplex: str) -> str:
    if y == YState.IDLE:
        return "Y=0: No Comm_Change is in progress."
    if y == YState.LOCAL_COMM_CHANGE:
        return "Y=1: Local was told to initiate a Physical Layer Comm_Change."
    if y == YState.SENDING_COMM_CHANGE:
        return "Y=2: Comm_Change directive is currently being sent across the link."
    if y == YState.WAIT_ACK:
        return "Y=3: Comm_Change directive sent; waiting for Comm_Change acknowledgement."
    if y == YState.RCCD_RECEIVED:
        if duplex.lower().startswith("half"):
            return "Y=4 is defined for full duplex only; in half duplex this condition is handled in other states."
        return "Y=4: Remote Comm_Change Directive (RCCD) has been received (full duplex only)."
    if y == YState.ACT_ON_RCCD:
        if duplex.lower().startswith("half"):
            return "Y=5 is defined for full duplex only; in half duplex acting on RCCD is covered elsewhere."
        return "Y=5: Acting upon the received RCCD (full duplex only)."
    return "Unknown Y value."

def describe_z(z: ZState) -> str:
    if z == ZState.INLOCK_TRUE:
        return "Z=0: SYMBOL_INLOCK_STATUS has NOT gone false yet during Comm_Change."
    if z == ZState.INLOCK_FALSE:
        return "Z=1: SYMBOL_INLOCK_STATUS has gone false at least once during Comm_Change."
    return "Unknown Z value."

def describe_modulation(mod: ModulationState) -> str:
    if mod == ModulationState.ON:
        return "Modulation=on: Coded symbols are being modulated onto the RF carrier (data/control frames can be sent)."
    else:
        return "Modulation=off: RF carrier may be present, but it is unmodulated (e.g., for acquisition/ranging/idle carrier)."

def describe_boolflag(name: str, val: BoolVar) -> str:
    if name == "NEED_PLCW":
        if val == BoolVar.TRUE:
            return "NEED_PLCW=true: A PLCW should be selected for output when possible."
        else:
            return "NEED_PLCW=false: The last PLCW has been selected for output; no immediate PLCW requirement."
    if name == "NEED_STATUS_REPORT":
        if val == BoolVar.TRUE:
            return "NEED_STATUS_REPORT=true: A status report should be selected for output when possible."
        else:
            return "NEED_STATUS_REPORT=false: A status report was just selected; no immediate status report requirement."
    return f"{name}={val.name.lower()}: generic boolean flag."


def describe_ids(remote_scid, recv_scid, recv_pcid):
    lines = []
    lines.append(f"REMOTE_SCID_BUFFER={remote_scid}: value used when testing frames whose ID field is interpreted as destination.")
    if recv_scid is None:
        lines.append("Receiving_SCID_Buffer=None: receiver will adopt the SCID from the first valid frame or be set by directive.")
    else:
        lines.append(f"Receiving_SCID_Buffer={recv_scid}: incoming frames are checked against this SCID for validation.")
    if recv_pcid is None:
        lines.append("Receiving_PCID_Buffer=None: receiver will adopt PCID from first valid frame or be set by directive.")
    else:
        lines.append(f"Receiving_PCID_Buffer={recv_pcid}: incoming frames are checked against this PCID (0 or 1).")
    return "\n".join(lines)


# -------- INTERACTIVE DEMO --------

def ask_int(prompt, allowed):
    while True:
        try:
            val = int(input(prompt))
            if val in allowed:
                return val
            print(f"Please enter one of {sorted(allowed)}.")
        except ValueError:
            print("Please enter a valid integer.")

def ask_choice(prompt, choices):
    choices_str = "/".join(choices)
    while True:
        val = input(f"{prompt} ({choices_str}): ").strip().lower()
        if val in [c.lower() for c in choices]:
            return val
        print(f"Please enter one of: {choices_str}.")

def main():
    print("=== Operational Control Variables Demo ===")
    print("This tool asks you for raw values and decodes them into human-readable states.\n")

    duplex = ask_choice("Link mode", ["full-duplex", "half-duplex"])

    # X, Y, Z
    x_val = ask_int("Enter X (0–5): ", set(range(0, 6)))
    y_val = ask_int("Enter Y (0–5): ", set(range(0, 6)))
    z_val = ask_int("Enter Z (0 or 1): ", {0, 1})

    # Modulation
    mod_choice = ask_choice("Modulation state", ["on", "off"])
    mod = ModulationState.ON if mod_choice == "on" else ModulationState.OFF

    # Flags
    plcw_choice = ask_choice("NEED_PLCW", ["true", "false"])
    need_plcw = BoolVar.TRUE if plcw_choice == "true" else BoolVar.FALSE

    status_choice = ask_choice("NEED_STATUS_REPORT", ["true", "false"])
    need_status = BoolVar.TRUE if status_choice == "true" else BoolVar.FALSE

    # ID buffers
    remote_scid = ask_int("REMOTE_SCID_BUFFER (0–1023): ", set(range(0, 1024)))

    def ask_optional_int(label, allowed):
        txt = input(f"{label} (enter blank for None, or value in {min(allowed)}–{max(allowed)}): ").strip()
        if txt == "":
            return None
        try:
            val = int(txt)
            if val in allowed:
                return val
        except ValueError:
            pass
        print("Invalid; treating as None.")
        return None

    recv_scid = ask_optional_int("Receiving_SCID_Buffer", set(range(0, 1024)))
    recv_pcid = ask_optional_int("Receiving_PCID_Buffer", {0, 1})

    # Decode
    x = XState(x_val)
    y = YState(y_val)
    z = ZState(z_val)

    print("\n=== Decoded Operational State ===")
    print(f"Link mode: {duplex}")

    print("\nSession Termination (X):")
    print(" ", describe_x(x, duplex))

    print("\nComm_Change (Y):")
    print(" ", describe_y(y, duplex))

    print("\nSYMBOL_INLOCK_STATUS (Z):")
    print(" ", describe_z(z))

    print("\nModulation:")
    print(" ", describe_modulation(mod))

    print("\nPLCW / Status Report Flags:")
    print(" ", describe_boolflag("NEED_PLCW", need_plcw))
    print(" ", describe_boolflag("NEED_STATUS_REPORT", need_status))

    print("\nID Buffers:")
    print(describe_ids(remote_scid, recv_scid, recv_pcid))

    # ---- VALIDITY WARNINGS BASED ON DUPLEX MODE ----
    print("\nValidity checks for chosen duplex mode:")

    if duplex.startswith("full"):
        # X=1 and X=3 are half-duplex-only in the spec
        if x == XState.LNMD_LOCAL or x == XState.GOT_RNMD_HAVE_DATA:
            print("  Warning: X=1 or X=3 are defined only for half-duplex; this combo would not occur in a true full-duplex link.")
        # Y=4 and Y=5 are allowed (full-duplex-only), so no warning here
    else:  # half-duplex
        # X=1 and X=3 are fine here
        # Y=4 and Y=5 are full-duplex-only in the spec
        if y in (YState.RCCD_RECEIVED, YState.ACT_ON_RCCD):
            print("  Warning: Y=4 and Y=5 are defined only for full-duplex; this combo would not occur in a true half-duplex link.")

    print("\nTip: Try switching between full- and half-duplex with the same X/Y to see which combinations are valid or flagged.")


if __name__ == "__main__":
    main()

=== Operational Control Variables Demo ===
This tool asks you for raw values and decodes them into human-readable states.


=== Decoded Operational State ===
Link mode: full-duplex

Session Termination (X):
  X=2: Local has accepted LNMD and is sending RNMD to the remote; termination will start once RNMD is exchanged.

Comm_Change (Y):
  Y=3: Comm_Change directive sent; waiting for Comm_Change acknowledgement.

SYMBOL_INLOCK_STATUS (Z):
  Z=1: SYMBOL_INLOCK_STATUS has gone false at least once during Comm_Change.

Modulation:
  Modulation=on: Coded symbols are being modulated onto the RF carrier (data/control frames can be sent).

PLCW / Status Report Flags:
  NEED_PLCW=true: A PLCW should be selected for output when possible.
  NEED_STATUS_REPORT=true: A status report should be selected for output when possible.

ID Buffers:
REMOTE_SCID_BUFFER=1: value used when testing frames whose ID field is interpreted as destination.
Receiving_SCID_Buffer=None: receiver will adopt the SCID from th